# MATH50003 (2022–23)
# Lab 1: Introduction to Julia

This problem sheet is designed to introduce some basic Julia
knowledge. Note each problem has multiple solutions, and the solution
sheet will show different ways of solving the same problem. We will discuss the
following:

1. Integers
2. Reals
2. Strings and parsing
3. Types

In assessment, _any_ "solution" will be accepted provided it does the right thing!
So you do not need to be able to write broadcasting or comprehensions
if you can do for loops.

We load the following packages:

In [2]:
using ColorBitstring, Test

## 1. Integers

Every primitive number type is stored as a sequence of bits.
The number of _bytes_ (i.e. 8-bits) can be deduced using the `sizeof` function:

In [3]:
sizeof(UInt32) # 4 bytes == 4*8 bits == 32 bits

4

The function `typeof` can be used to determine the type of a number.
By default when we write an integer (e.g. `-123`) it is of type `Int`:

In [4]:
typeof(5)

Int64

-----
**Problem 1.1** Use `sizeof` to determine how many bits your machine uses for the type `Int`.

In [5]:
sizeof(Int)

8

-----

There are a few ways to create other types of integers. Conversion
converts between different types:

In [5]:
UInt8(5) # converts an `Int` to an `UInt8`, displaying the result in hex

0x05

This fails if a number cannot be represented as a specified type: e.g. `UInt8(-5)` and `UInt8(2^8)`.

(These can also be written as e.g. `convert(UInt8, 5)`.)
We can also create unsigned integers by specifying their bits
by writing `0b` followed by a sequence of bits:

In [6]:
0b101 # isa UInt8, the smallest type with at least 3 bits

0x05

In [7]:
0b10111011101 # isa UInt16, the smallest type with at least 11 bits

0x05dd

Or in base-16 using hexadecimal format (with digits `0–9a–f` following
an `0x`), where each digit takes 4 bits to represent (since $2^4 = 16$):

In [8]:
0xabcde # isa UInt32, the smallest type with at least 4*5 = 20 bits

0x000abcde

-----
**Problem 1.2** Use binary format to create an `Int` corresponding to $(101101)_2$.

In [6]:
0b101101

0x2d

-----

**Problem 1.3** What happens if you specify more than 64 bits using `0b⋅⋅…⋅⋅`?
What if you specify more than 128 bits?

In [9]:
0b11111111111111111111111111111111111111111111111111111111111111111

0x0000000000000001ffffffffffffffff

-----

We can also reinterpret a sequence of bits in a different format:

In [11]:
reinterpret(Int8, 0b11111111) # Create an Int8 with the bits 11111111

-1

Arithmetic follows modular arithmetic. The following examples test your understanding of this.

-----

**Problem 1.5** Can you predict what the output of the following will be before hitting return?

In [14]:
Int(UInt8(120) + UInt8(10)) # Convert to `Int` to see the number printed in decimal

130

In [15]:
Int8(120) + Int8(10)

-126

In [17]:
UInt8(2)^7

0x80

In [18]:
Int8(2)^7

-128

In [19]:
Int8(2)^8

0

-----

## 2. Reals

Real numbers interpret a sequence of bits in floating point format.

-----
**Problem 2.1** Use `printbits` to guess the binary representation of $1/5$.

In [20]:
printbits(1/5)

0011111111001001100110011001100110011001100110011001100110011010

-----

**Problem 2.2** Create a positive `Float16` whose exponent is $q = 156$ and has significand
bits
$$
b_k = \begin{cases}
    1 & k\hbox{ is prime} \\
    0 & \hbox{otherwise}
    \end{cases}
$$

In [26]:

printbits(reinterpret(Float16, 0b0111000110101000))

0111000110101000

-----

**Problem 2.3** Create the smallest postive non-zero sub-normal `Float16` by specifying
its bits.

In [32]:
reinterpret(Float16, 0b0000000000000001)

Float16(6.0e-8)

-----

## 3. Strings and parsing

Strings are a convenient way of representing arbitrary strings of digits.
For example we can convert bits of a number to a string of "1"s and "0"s using the function `bitstring`.

-----

**Problem 3.1** Can you predict what the output of the following will be before hitting return?

In [33]:
bitstring(11) # Semi-colon prohibits output, delete to check your answer

"0000000000000000000000000000000000000000000000000000000000001011"

In [34]:
bitstring(-11)

"1111111111111111111111111111111111111111111111111111111111110101"

-----

We can `parse` a string of digits in base 2 or 10:

In [22]:
parse(Int8, "11"; base=2),
parse(Int8, "00001011"; base=2)

(3, 11)

Be careful with "negative" numbers, the following will fail: `parse(Int8, "10001011"; base=2)`

It treats the string as binary digits, NOT bits. That is, negative numbers
are represented using the minus sign:

In [23]:
parse(Int8, "-00001011"; base=2)

-11

-----

**Problem 3.2** Combine `parse`, `reinterpret`, and `UInt8` to convert the
above string to a (negative) `Int8` with the specified bits.

In [37]:
reinterpret(Int8, parse(UInt8, "10001011"; base=2))

-117

-----

To concatenate strings we use `*` (multiplication is used because string concatenation
is non-commutative):

In [25]:
"hi" * "bye"

"hibye"

The string consisting of the first nine characters can be found using `str[1:9]` where `str` is any string:

In [26]:
str="hibye0123445556"
str[1:9]  # returns "hibye0123"

"hibye0123"

The string consisting of the 11th through last character can be found using `str[11:end]`:

In [27]:
str="hibye0123445556"
str[11:end]  # returns "45556"

"45556"

-----

**Problem 3.3** Complete the following function that sets the 10th bit of an `Int32` to `1`,
and returns an `Int32`, assuming that the input is a positive integer, using `bitstring`,
`parse` and `*`.

In [50]:
function tenthbitto1(x::Int32)
    x_string = bitstring(x)
    proper_string = x_string[1:9] * "1" * x_string[11:end]
    parse(Int32, proper_string; base=2)
end

# unit tests are to help you check your result
# Change to `@test` to see if your test passes
@test tenthbitto1(Int32(100)) ≡ Int32(4194404)

Test Passed

-----

**Problem 3.4**  Modify the previous function to also work with negative numbers.

In [55]:
function tenthbitto1(x::Int32)
    x_string = bitstring(x)
    proper_string = x_string[1:9] * "1" * x_string[11:end]
    reinterpret(Int32, parse(UInt32, proper_string; base=2))
end

@test tenthbitto1(Int32(100)) ≡ Int32(4194404)
@test tenthbitto1(-Int32(100000010)) ≡ Int32(-95805706)

Test Passed

-----

## 4. Types

Types allow for combining multiple numbers (or other types) to represent a more complicated
object. That is, while a computer can only apply functions on $p$-bits at a time,
these functions can be combined to perform more complicated operations on types
that require more than $p$-bits. A simple example is a rational function.

-----

**Problem 4.1** Create a type `Rat` with two `Int` fields, `p` and `q`,
representing a Ratiional function including `+`, `*`, `-`, and `/`.

In [80]:
# `struct` creates a new type called `Rat`
# consisting of 128 bits, half encode `p`
# and half encode `q`
struct Rat
    p::Int
    q::Int
end

# A new instance of `Rat` is created via e.g. `Rat(1, 2)` represents 1/2
# where the first argument specifies `p` and the second argument `q`.
# The fields are accessed by `.`:

x = Rat(1, 2)
@test x.p == 1
@test x.q == 2

# We import `+`, `-`, `*`, `/` so we can "overload" these operations specifically
# for `Rat`.
import Base: +, -, *, /, ==


# The ::Rat means the following version of `==` is only called if both arguments
# are Rat
function ==(x::Rat, y::Rat)
    # TODO: implement equality, making sure to check the case where
    # the numerator/denominator are possibly reducible
    # Hint: `gcd` and `div` may be useful. Use `?` to find out what they do
    yp = y.p
    yq = y.q
    xp = Int(x.p)
    xq = Int(x.q)
    if gcd(xp, xq) != 1
        g = gcd(xp, xq)
        xp /= g
        xq /= g
    end
    if gcd(yp, yq) != 1
        g = gcd(yp, yq)
        yp /= g
        yq /= g
    end
    if xp == yp && xq == yq
        return true
    else
        return false
    end
end

# We can also support equality when `x isa Rat` and `y isa Integer`
function ==(x::Rat, y::Integer)
    if gcd(x.p, x.q) != x.q
        return false
    elseif x.p / x.q != y
        return false
    else
        return true
    end
end

# TODO: implement ==(x::Integer, y::Rat)
function ==(x::Integer, y::Rat)
    return y == x
end

@test Rat(1, 2) == Rat(2, 4)
@test Rat(1, 2) ≠ Rat(1, 3)
@test Rat(2,2) == 1
@test 1 == Rat(2,2)

# TODO: implement +, -, *, and /,
function +(x::Rat, y::Rat)
    if x.q == y.q
        return Rat(x.p + y.p, x.q)
    else
        l = lcm(x.q, y.q)
        xp = Int(x.p * (l/x.q))
        yp = Int(y.p * (l/y.q))
        return Rat(xp + yp, l)
    end
end

function -(x::Rat, y::Rat)
    return x + Rat(-y.p, y.q)
end

function *(x::Rat, y::Rat)
    zp = x.p * y.p
    zq = x.q * y.q
    g = gcd(zp, zq)
    zp /= g
    zq /= g
    return Rat(zp, zq)
end

function /(x::Rat, y::Rat)
    return x * Rat(y.q, y.p)
end

@test Rat(1, 2) + Rat(1, 3) == Rat(5, 6)
@test Rat(1, 3) - Rat(1, 2) == Rat(-1, 6)
@test Rat(2, 3) * Rat(3, 4) == Rat(1, 2)
@test Rat(2, 3) / Rat(3, 4) == Rat(8, 9)

Test Passed

---------

Templating is a way of letting fields take on different types. For example, the following
code allows `x` to be any type:

In [1]:
struct Foo{T}
    x::T
end

Foo(5) # isa Foo{Int}

Foo{Int64}(5)

In [2]:
Foo("hi") # isa Foo{String}

Foo{String}("hi")

-----

**Problem 4.2** Modify the above code so that `p` and `q` can be other types but both the same type,
for example, `Int16` or `BigInt`.

In [90]:
# `struct` creates a new type called `Rat`
# consisting of 128 bits, half encode `p`
# and half encode `q`
struct Rati{T}
    p::T
    q::T
end

# A new instance of `Rat` is created via e.g. `Rat(1, 2)` represents 1/2
# where the first argument specifies `p` and the second argument `q`.
# The fields are accessed by `.`:

x = Rati(1, 2)
@test x.p == 1
@test x.q == 2

# We import `+`, `-`, `*`, `/` so we can "overload" these operations specifically
# for `Rat`.
import Base: +, -, *, /, ==


# The ::Rat means the following version of `==` is only called if both arguments
# are Rat
function ==(x::Rati, y::Rati)
    # TODO: implement equality, making sure to check the case where
    # the numerator/denominator are possibly reducible
    # Hint: `gcd` and `div` may be useful. Use `?` to find out what they do
    yp = y.p
    yq = y.q
    xp = x.p
    xq = x.q
    if gcd(xp, xq) != 1
        g = gcd(xp, xq)
        xp /= g
        xq /= g
    end
    if gcd(yp, yq) != 1
        g = gcd(yp, yq)
        yp /= g
        yq /= g
    end
    if xp == yp && xq == yq
        return true
    else
        return false
    end
end

# We can also support equality when `x isa Rat` and `y isa Integer`
function ==(x::Rati, y::Integer)
    if gcd(x.p, x.q) != x.q
        return false
    elseif x.p / x.q != y
        return false
    else
        return true
    end
end

# TODO: implement ==(x::Integer, y::Rat)
function ==(x::Integer, y::Rati)
    return y == x
end

@test Rati(1, 2) == Rati(2, 4)
@test Rati(1, 2) ≠ Rati(1, 3)
@test Rati(2,2) == 1
@test 1 == Rati(2,2)

# TODO: implement +, -, *, and /,
function +(x::Rati, y::Rati)
    if x.q == y.q
        return Rati(x.p + y.p, x.q)
    else
        l = lcm(x.q, y.q)
        xp = Int(x.p * (l/x.q))
        yp = Int(y.p * (l/y.q))
        return Rati(xp + yp, l)
    end
end

function -(x::Rati, y::Rati)
    return x + Rati(-y.p, y.q)
end

function *(x::Rati, y::Rati)
    zp = x.p * y.p
    zq = x.q * y.q
    g = gcd(zp, zq)
    zp /= g
    zq /= g
    return Rati(zp, zq)
end

function /(x::Rati, y::Rati)
    return x * Rati(y.q, y.p)
end

@test Rati(1, 2) + Rati(1, 3) == Rati(5, 6)
@test Rati(1, 3) - Rati(1, 2) == Rati(-1, 6)
@test Rati(2, 3) * Rati(3, 4) == Rati(1, 2)
@test Rati(2, 3) / Rati(3, 4) == Rati(8, 9)

Error During Test at In[90]:101
  Test threw exception
  Expression: Rati(2, 3) * Rati(3, 4) == Rati(1, 2)
  MethodError: no method matching typemax(::Type{Integer})
  Closest candidates are:
    typemax(::Union{Dates.DateTime, Type{Dates.DateTime}}) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/types.jl:453
    typemax(::Union{Dates.Date, Type{Dates.Date}}) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/types.jl:455
    typemax(::Union{Dates.Time, Type{Dates.Time}}) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/types.jl:457
    ...
  Stacktrace:
    [1] tryparse_internal(#unused#::Type{Integer}, s::String, startpos::Int64, endpos::Int64, base_::Int64, raise::Bool)
      @ Base ./parse.jl:128
    [2] parse(::Type{Integer}, s::String; base::Int64)
      @ Base ./parse.jl:241
    [3] *(x::Rati{Int64}, y::Rati{Int64})
      @ Main ./In[90]:89
    [4] top-level

LoadError: [91mThere was an error during testing[39m

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*